# Chosing the Image DPI for Performance

In [69]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [70]:
df = pd.read_csv("../datasets/cmyk-by-dpi.csv")
df['sum'] = df['c'] + df['m'] + df['y'] + df['k']
df.drop(columns=['page', 'processing_time', 'c', 'm', 'y', 'k'], inplace=True)

In [60]:
# Create dataframe for each dpi
df5 = df[df['dpi'] == 5].copy().drop(columns=['dpi']).reset_index(drop=True)

df10 = df[df['dpi'] == 10].copy().drop(columns=['dpi']).reset_index(drop=True)

df20 = df[df['dpi'] == 20].copy().drop(columns=['dpi']).reset_index(drop=True)

df50 = df[df['dpi'] == 50].copy().drop(columns=['dpi']).reset_index(drop=True)

df100 = df[df['dpi'] == 100].copy().drop(columns=['dpi']).reset_index(drop=True)

df150 = df[df['dpi'] == 150].copy().drop(columns=['dpi']).reset_index(drop=True)

df200 = df[df['dpi'] == 200].copy().drop(columns=['dpi']).reset_index(drop=True)

df250 = df[df['dpi'] == 250].copy().drop(columns=['dpi']).reset_index(drop=True)

In [61]:
scaler = StandardScaler()
kmeans = KMeans(n_clusters=6, n_init='auto')

kmeans.fit(scaler.fit_transform(df20[['sum']]))

KMeans(n_clusters=6, n_init='auto')

In [62]:
df50['label'] = kmeans.labels_

In [63]:
y_test = pd.read_csv("../datasets/300dpi-6-cluster-with-label.csv")
y_test.drop(columns=['Unnamed: 0', 'sum'], inplace=True)
y_test

,label
0,3
1,2
2,0
3,2
4,2
...,...
847,2
848,2
849,2
850,2


In [64]:
true_count = 0
for pred, test in zip(df50['label'], y_test['label']):
    if pred == test:
        true_count +=1

accuracy = true_count / len(y_test) * 100

In [65]:
accuracy

1.8779342723004695

In [86]:
import pickle

kmeans, scaler = pickle.load(open('../models/kmeans_and_scaler.pkl', 'rb'))

df = pd.read_csv("../datasets/cmyk-by-dpi.csv")
df['sum'] = df['c'] + df['m'] + df['y'] + df['k']
df.drop(columns=['page', 'processing_time', 'c', 'm', 'y', 'k'], inplace=True)

y_test = pd.read_csv("../datasets/300dpi-6-cluster-with-label.csv")
y_test.drop(columns=['Unnamed: 0', 'sum'], inplace=True)

dpi_list =  [5, 10, 20] + list(range(50, 301, 50))

accuracy = {}
for dpi in dpi_list:
    temp_df = df[df['dpi'] == dpi].copy().drop(columns=['dpi']).reset_index(drop=True)
    kmeans.predict(scaler.transform(temp_df[['sum']]))
    temp_df['label'] = kmeans.labels_

    true_count = 0
    for pred, test in zip(temp_df['label'], y_test['label']):
        if pred == test:
            true_count +=1
            
    accuracy[dpi] = true_count / len(y_test) * 100

print(accuracy)

{5: 100.0, 10: 100.0, 20: 100.0, 50: 100.0, 100: 100.0, 150: 100.0, 200: 100.0, 250: 100.0, 300: 100.0}


Manarik sekali! dengan dataset 300dpi, accuracy untuk data di semua DPI tetap 100%. Karena 5dpi memiliki waktu konversi paling cepat dengan accuracy yang tetap 100%. Maka implementasi model pada aplikasi akan menggunakan konversi ke 5dpi.

,sum
0,154.52
1,0.08
2,15.71
3,2.99
4,4.32


In [93]:
df = pd.read_csv("../datasets/cmyk-by-dpi.csv")
df['sum'] = df['c'] + df['m'] + df['y'] + df['k']
df.drop(columns=['processing_time', 'c', 'm', 'y', 'k'], inplace=True)

In [97]:
df[(df['dpi'].isin([5,10,50,100,200,300])) & (df['page'] == 1)]

,page,dpi,sum
0,1,300,157.95
1704,1,200,157.92
3408,1,100,157.75
4260,1,50,157.37
5964,1,10,154.52
6816,1,5,152.05
